In [44]:
!pip install av
!pip install facenet_pytorch
!pip install ffmpeg-python
!pip install matplotlib

In [1]:
import os
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
import torch.optim as optim
import torchvision.transforms as transforms
import cv2
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
import facenet_pytorch
import matplotlib.pyplot as plt
import ffmpeg
import torchvision.io as tio
import av
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.transforms import functional as F
from torchvision import transforms
import math

c:\Users\Asus\Desktop\Phosphene.AI\sizeinvar_timesformer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import zipfile
import os

#zip_file = "/content/drive/MyDrive/Phosphene/sample_data.zip"
#with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#        zip_ref.extractall()

In [3]:
train_dir = "sample_data/target/train"
val_dir = "sample_data/target/val"
train_mask_dir = "sample_data/target_mask/train"
val_mask_dir = "sample_data/target_mask/val"

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
])

In [5]:
import random

class VideoDataset(Dataset):
    def __init__(self, directory, target_dir, transform=None, frame_rate=24, pick_only=24):
        self.directory = directory
        self.target_dir = target_dir
        self.transform = transform
        self.frame_rate = frame_rate
        self.pick_only = pick_only

        self.videos = sorted(os.listdir(self.directory))
        self.masks = sorted(os.listdir(self.target_dir))

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video_path = os.path.join(self.directory, self.videos[idx])
        mask_path = os.path.join(self.target_dir, self.masks[idx])
    
        video_frames, mask_frames = self.extract_frames(video_path, mask_path)
        
        if self.transform:
            video_frames = [self.transform(frame) for frame in video_frames]
            mask_frames = [self.transform(frame) for frame in mask_frames]

        video_frames = torch.stack(video_frames).permute(1, 0, 2, 3)
        mask_frames = torch.stack(mask_frames).permute(1, 0, 2, 3)

        return video_frames, mask_frames

    def extract_frames(self, video_path, mask_path):
        cap_video = cv2.VideoCapture(video_path)
        cap_mask = cv2.VideoCapture(mask_path)
        frame_count = int(cap_video.get(cv2.CAP_PROP_FRAME_COUNT))
        desired_frame_count = self.pick_only
        
        # Pick a random starting point within the video 
        start_idx = random.randint(0, max(0, frame_count - desired_frame_count))
        
        video_frames = []
        mask_frames = []
        
        cap_video.set(cv2.CAP_PROP_POS_FRAMES, start_idx)  
        cap_mask.set(cv2.CAP_PROP_POS_FRAMES, start_idx)  
        
        for i in range(desired_frame_count):
            ret_video, frame_video = cap_video.read()
            ret_mask, frame_mask = cap_mask.read()
            if not (ret_video and ret_mask):
                break
            frame_video = cv2.cvtColor(frame_video, cv2.COLOR_BGR2RGB)
            frame_mask = cv2.cvtColor(frame_mask, cv2.COLOR_BGR2GRAY)
            video_frames.append(frame_video)
            
            mask_frames.append(frame_mask)
           
        
        cap_video.release()
        cap_mask.release()
        return video_frames, mask_frames


In [6]:
train_dataset = VideoDataset(train_dir, target_dir=train_mask_dir, transform=transform)
val_dataset = VideoDataset(val_dir, target_dir=val_mask_dir,transform=transform)

batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
for videos, masks in train_loader:
    print("Video batch shape:", videos.shape)
    print("Mask batch shape:", masks.shape)
    break

Loading video: sample_data/target/train\train_00000409.mp4
Loading mask: sample_data/target_mask/train\train_00000409_mask.mp4
starting index: 52
Video batch shape: torch.Size([1, 3, 24, 224, 224])
Mask batch shape: torch.Size([1, 1, 24, 224, 224])


In [9]:
len(train_dataset),len(val_dataset)

(517, 102)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=2, stride=2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(64, 1, kernel_size=2, stride=2)
        )

    def forward(self, x):
        # Encoder
        x1 = self.encoder(x)

        # Decoder
        x = self.decoder(x1)
        x = torch.sigmoid(x)

        return x


model = UNet()


In [11]:
model = model.to(device)
print(model)

UNet(
  (encoder): Sequential(
    (0): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU(inplace=True)
    (2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU(inplace=True)
    (2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (3): ReLU(inplace=True)
    (4): ConvTranspose3d(64, 1, kernel_size=(2, 2, 2), stride=(2, 2, 2))
  )
)


In [12]:

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
print(type(model))
print(dir(model))

<class '__main__.UNet'>
['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register

In [ ]:
def dice_coefficient(y_true,y_pred):
    smooth = 1e-6
    intersection = torch.sum(y_true*y_pred)
    union = torch.sum(y_true) + torch.sum(y_pred)
    dice = (2. * intersection + smooth)/(union+smooth)
    return dice


In [ ]:
for epoch in range(5):
    model.train()
    epoch_loss = 0.0
    similarity_index = 0.0
    total_samples = 0
    
    for batch_idx, (videos, masks) in enumerate(train_loader):
        print(f"Epochs : {epoch+1}")
        videos, masks = videos.to(device), masks.to(device)
        print(f"Batch {batch_idx+1}/{len(train_loader)}")
        print(f"Video shape: {videos.shape}")
        print(f"Mask shape: {masks.shape}")

        optimizer.zero_grad()
        outputs = model(videos)
        print(f"Output shape: {outputs.shape}")

        loss = criterion(outputs, masks)
        print(f"Loss for batch {batch_idx+1}: {loss.item():.4f}")

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        batch_size = videos.size(0)
        total_samples += batch_size
        
        #similarity index 
        outputs_binary = torch.sigmoid(outputs) > 0.5
        similarity_index += dice_coefficient(masks, outputs_binary).item() * batch_size

    avg_loss = epoch_loss / len(train_loader)
    avg_similarity_index = similarity_index / total_samples
    
    print(f'Epoch [{epoch+1}/{5}], Average Loss: {avg_loss:.4f}, Similarity Index: {avg_similarity_index:.4f}')

   

In [ ]:
model.eval()
val_loss = 0.0
val_iou = 0.0
total_val_samples = 0

        
with torch.no_grad():
    for batch_idx, (videos, masks) in enumerate(val_loader):
        videos, masks = videos.to(device), masks.to(device)
        outputs = model(videos)
        loss = criterion(outputs, masks)
        val_loss += loss.item()
            
        
            
        total_val_samples += videos.size(0)
    
avg_val_loss = val_loss / len(val_loader)
    
print(f'Validation Loss: {avg_val_loss:.4f}')


In [ ]:
EPOCH = epoch
PATH = "model.pt"
LOSS = val_loss

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

In [ ]:
for videos, masks in train_loader:
      videos, masks = videos.to(device), masks.to(device)
      break
plt.imshow(transforms.ToPILImage()(outputs[0,:,3,:,:]))

In [ ]:
vid = outputs.cpu().permute(0,2,3,4,1).numpy()[0]
plt.imshow(vid[0,:,:,:])

In [ ]:
outputs.max()